In [ ]:
!git clone https://github.com/team21-2023/Team_21_InterIIT.git
!mv Team_21_InterIIT data
!mv data/* .
!rm -rf data

Cloning into 'Team_21_InterIIT'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 0), reused 9 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), 3.60 MiB | 8.72 MiB/s, done.


In [ ]:
!pip install -U -q sentence-transformers
#!pip install -q gensim==4.1.2
#!pip install elasticsearch==7.9.1
#!pip install -q sentencepiece
#!pip install -q transformers 
#!pip install -q datasets optimum onnx onnxruntime onnxruntime_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 102.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 106.3 MB/s eta 0:00:00


In [ ]:
# Allowed to make changes.
import collections
import json
import string
import timeit

# Allowed to make changes.
import collections
import string
import timeit

#imports for data processing
import pandas as pd
import numpy as np
import json
from ast import literal_eval
import re
from tqdm import tqdm
from google.colab import files
from numpy import genfromtxt
from functools import reduce

#imports for modelling
import os
import sys
import time
import torch
import h5py
import pickle
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

In [ ]:
paragraphs_data = pd.read_csv("paragraphs.csv", encoding="utf-8")
themes = paragraphs_data.theme.unique()
theme_weights = {i:1/len(themes) for i in themes}
#Adding theme index
theme_to_id_dict = {}
for i in range(len(themes)):
    t = themes[i]
    theme_to_id_dict[t]=i
paragraphs_data['tid'] = paragraphs_data.apply(lambda row: theme_to_id_dict[row.theme], axis=1)

In [ ]:
paragraphs_data.head()

,id,paragraph,theme,tid
0,1,The iPod is a line of portable media players a...,IPod,0
1,2,"Like other digital music players, iPods can se...",IPod,0
2,3,Apple's iTunes software (and other alternative...,IPod,0
3,4,"Before the release of iOS 5, the iPod branding...",IPod,0
4,5,"In mid-2015, a new model of the iPod Touch was...",IPod,0


In [ ]:
def create_p_embedding_dl_ir(model, embedding_file_name, is_model_quantized=False):
  p_embed_h5f = h5py.File(embedding_file_name, 'w')
  for tid in paragraphs_data.tid.unique():
    #theme based data
    t_paragraphs_data = paragraphs_data[paragraphs_data.tid==tid]
    t_para = t_paragraphs_data.paragraph.to_list()
    #encoding passages
    if not is_model_quantized:
      t_para_embedding = model.encode(t_para)
    else:
      t_para_embedding = model(t_para)
      t_para_embedding = reduce(lambda x,y: np.concatenate((x,y), axis=0), [np.array(t_para_embedding[i]) for i in range(len(t_para_embedding))])
    p_embed_h5f.create_dataset(f'theme_{tid}', data=t_para_embedding)
  p_embed_h5f.close()

In [ ]:
#IR DL model config
dl_ir_model_name = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
#vanilla model
dl_ir_model = SentenceTransformer(dl_ir_model_name)

Encoding paragraphs using DL IR model

In [ ]:
#IR DL model config
if not os.path.exists('passage_embedding_dl_ir.h5'):
  execution_time = timeit.timeit(lambda: create_p_embedding_dl_ir(dl_ir_model, 'passage_embedding_dl_ir.h5'), number=1)
  print(f"Time taken to encode {len(paragraphs_data)} paragraphs is {execution_time} i.e {execution_time/len(paragraphs_data)} seconds per paragraph")
  files.download('passage_embedding_dl_ir.h5')

Time taken to encode 1179 paragraphs is 2256.770050229 i.e 1.9141391435360475 seconds per paragraph


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>